<h1> Project Overview </h1>
<p> from conversation 12/1/2016
Robert wants to see installations over time for PG&E, SCE, SDG&E
 <br/>- average size and number of installations 
 <br/>- vendors installation (pie chart) 
</p>
<p> Source::  https://www.californiasolarstatistics.ca.gov/data_downloads/ </p>


<h1> Data Clean Up Scripts </h1>
<p> Run this script to load orignial data set and then output new file "filtered_data.xlsx" that is then loaded into the next script "analysis.py"</p>

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import datetime as datetime
import scipy as scipy
from scipy.stats import mode
from bokeh.charts import Bar, output_file, show


<h2>Analysis of NEM Current Interconnections</h2>

In [2]:
filename = input('Enter File Name of most up-to-date NEM dataset:')

Enter File Name of most up-to-date NEM dataset:NEM_CurrentlyInterconnectedDataset_2016-12-31.csv


In [4]:
#filename = 'NEM_CurrentlyInterconnectedDataset_2016-11-30.csv'
NEM_data=pd.read_csv(('..//Data//'+filename),low_memory=False)

<h3> Clean Up data: </h3>

In [5]:
pd.options.display.float_format = '{:,.2f}'.format
NEM_df = pd.DataFrame(NEM_data)
NEM_df[:4]

#top installers determined visually looking at: 
NEM_df['Installer Name'].value_counts()[:30]

#Could also use a dictionary (can save in a seperate text file)
#{'SOLARCITY': ['Solar City', 'SolarCity']}

TopInstallers = ['SOLARCITY','VIVINT','VERENGO','SUNPOWER','SUNRUN','REC SOLAR','BAKER ELECTRIC','SULLIVAN','SUNGEVITY']
#Combine Installer Names (data has repeats like "Solar City" vs "Solar City Incorporation")
# Or inconsistent Values /format (upper case vs lower case)
for i in range(0,len(NEM_df['Installer Name'])):
    
    #first format to Uppercase for consistency
    installer = str(NEM_df.get_value(i,'Installer Name')).upper()
    NEM_df.set_value(i,'Installer Name',installer)
    
    #next, combine with others (if one of top vendors: Solar City,  Vivint, Verengo, Sunpower, Sunrun, REC Solar, Baker Electric, Sullivan, Sungevity)
    for co in TopInstallers:
        if co in installer:
            NEM_df.set_value(i,'Installer Name',co)

            
NEM_df.to_csv(('..\\Data\\cleaned'+filename))

NEM_df['Utility'].unique()

#first set of filters: any data that is installed
filter_df = NEM_df[NEM_df['Application Status']=='Interconnected']

#Remove NEM_df to free up memory
del NEM_df

#then use completed date
date_S = pd.to_datetime(filter_df['App Complete Date'])
filter_df['App Complete Date'] = date_S
endT = max(date_S)
startT = min(date_S)
#filter_df.index = date_S

#add columns to refer to year and month:
filter_df['Year'] = filter_df['App Complete Date'].dt.year
filter_df['Month'] = filter_df['App Complete Date'].dt.month

#remove columns that provide no real value, lots of NAN (require at least 10 non-NA values)
filter_df.dropna(axis=1,thresh=100).columns
drop_cols = ['Module Manufacturer 1', 'Module Quantity 1', 'Module Model 2',
       'Module Manufacturer 2', 'Module Quantity 2', 'Module Model 3',
       'Module Manufacturer 3', 'Module Quantity 3', 'Module Model 4',
       'Module Manufacturer 4', 'Module Quantity 4', 'Module Model 5',
       'Module Manufacturer 5', 'Module Quantity 5', 'Module Model 6',
       'Module Manufacturer 6', 'Module Quantity 6', 'Module Model 7',
       'Module Manufacturer 7', 'Module Quantity 7', 'Module Model 8',
       'Module Manufacturer 8', 'Module Quantity 8', 'Module Model 9',
       'Module Manufacturer 9', 'Module Quantity 9', 'Module Model 10',
       'Module Manufacturer 10', 'Module Quantity 10', 'Inverter Model 1',
       'Inverter Manufacturer 1', 'Inverter Quantity 1', 'Inverter Model 2',
       'Inverter Manufacturer 2', 'Inverter Quantity 2', 'Inverter Model 3',
       'Inverter Manufacturer 3', 'Inverter Quantity 3', 'Inverter Model 4',
       'Inverter Manufacturer 4', 'Inverter Quantity 4', 'Inverter Model 5',
       'Inverter Manufacturer 5', 'Inverter Quantity 5', 'Inverter Model 6',
       'Inverter Manufacturer 6', 'Inverter Quantity 6', 'Inverter Model 7',
       'Inverter Manufacturer 7', 'Inverter Quantity 7', 'Inverter Model 8',
       'Inverter Manufacturer 8', 'Inverter Quantity 8', 'Inverter Model 9',
       'Inverter Manufacturer 9', 'Inverter Quantity 9', 'Inverter Model 10',
       'Inverter Manufacturer 10', 'Inverter Quantity 10', 'Inverter Model 11',
       'Inverter Manufacturer 11', 'Inverter Quantity 11', 'Inverter Model 12',
       'Inverter Manufacturer 12', 'Inverter Quantity 12']

In [6]:
filter_df = filter_df.drop(drop_cols,axis=1)
#remove duplicates (using key fields:)
len_b4 = len(filter_df)
    filter_df = filter_df.drop_duplicates(['System Size DC','Application Id','Utility','App Complete Date','Service Zip'])
len_af = len(filter_df)
total_dups_dropped = len_b4 - len_af
filter_df = filter_df.dropna(axis=0,how='any',subset=['System Size DC'])
total_na_dropped = len_af - len(filter_df)

In [8]:
print ('Total NA entries dropped:  ',total_na_dropped)
print ('TOtal duplicates dropped:  ',total_dups_dropped)

Total NA entries dropped:   12105
TOtal duplicates dropped:   0


In [ ]:
#need to remove any installation over 1000 kw (!MW) which violates NEM criterea
filter_df = filter_df[filter_df['System Size DC']<1000]

In [ ]:
filter_df['System Size DC'].hist(bins=100)

In [ ]:
#Remove erroneous data (.5% quantile)
hi = filter_df['System Size DC'].quantile(.995)
low = filter_df['System Size DC'].quantile(.005)
filter_df = filter_df[filter_df['System Size DC']<hi
filter_df = filter_df[filter_df['System Size DC']>low
filter_df['System Size DC'].hist(bins=100)


In [ ]:
filter_df.to_excel('filtered_data.xlsx')